#### Instantiate llama2 llm with langchain Ollama

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2", temperature=0.01)

In [2]:
print(llm.invoke('Tell me a joke'))


Sure, here's one:

Why don't scientists trust atoms?
Because they make up everything!

I hope that brought a smile to your face!


#### Define Class to use the invoke method to assess whether statement is relevant to a question

###### Old prompts

In [ ]:
def _generate_prompt0(self, question: str, statement: str) -> str:
        return f'''
            The statement below is from a reddit thread for employees of the company for which a question is asked. 
            Does the statement provide information relevant to the question listed below (delimeted by ```)? 
            Please format the output as a dictionary with the following keys: "relevant", "reason". 
            Relevant should be a boolean value indicating whether the statement is relevant to the question.
            Reason should be a string explaining why and how the statement is or is not relevant.
            Two examples and accompanying outputs are provided here (delimited by ~~~):
            ~~~
            Example 1:
            Question: "What team has won the most Super Bowls?"
            Statement: "The New England Patriots have won the Super Bowl a record six times"
            Output: {{
                "relevant": true,
                "reason": "The statement provides about the team that has won the most Super Bowls"
            }}
            ~~~
            Example 2:
            Question: "What team has won the most Super Bowls?"
            Statement: "I do not find that interesting"
            Output: {{
                "relevant": false,
                "reason": "The statement provided does not have the needed context to help answer the question"
            }}
            ~~~
            ```
            Question: {question}
            ```
            Statement:{statement}
            ```
            '''.replace('            ','')

def prompt_template(question: str, statement: str) -> str:
    return f'''
        Your role is to label chunks of texts to determine whether they are relevant to a given question.
        The statement provided is taken from employees of the company for which a question is asked. 
        A statement should be labeled as relevant (true) if it provides information that helps answer the question.
        It does not need to answer the question directly, but it should provide at least something relevant to the question.
        For example, if the question is "What do employees have to say about promotions?", a statement like 
        "I have never gotten a raise" would be labeled as relevant.
        Does the statement provide information relevant to the question listed below (delimeted by ```)? 
        Please format the output as a dictionary with the following keys: "relevant", "reason". 
        Relevant should be a boolean value indicating whether the statement is relevant to the question.
        Reason should be a string explaining why and how the statement is or is not relevant.
        An examples and accompanying output is provided here (delimited by ~~~):
        ~~~
        Example:
        Question: "What team has won the most Super Bowls?"
        Statement: "I do not find that interesting"
        Output: {{
            "relevant": false,
            "reason": "The statement provided does not have the needed context to help answer the question"
        }}
        ~~~
        ```
        Question: {question}
        ```
        Statement:{statement}
        ```
        '''.replace('        ','')

def _generate_prompt(self, question: str, statement: str) -> str:
        return f'''
            Your role is to label chunks of text (statements) to determine whether they are relevant to a given question for a retrieval system.
            The statement provided is taken from employees of the company for which a question is asked. 
            A statement should be labeled as relevant (true) if it provides information that helps answer the question.
            It does not need to answer the question directly, but it should provide at least something relevant to the question.
            For example, if the question is "What do employees have to say about promotions?", a statement like 
            "I have never gotten a raise" would be labeled as relevant.
            Does the statement provide information relevant to the question listed below (delimeted by ```)? 
            Please format the output as a dictionary with the following keys: "relevant", "reason". 
            Relevant should be a boolean value indicating whether the statement is relevant to the question.
            Reason should be a string explaining why and how the statement is or is not relevant.
            An examples and accompanying output is provided here (delimited by ~~~):
            ~~~
            Example:
            Question: "What team has won the most Super Bowls?"
            Statement: "I do not find that interesting"
            Output: {{
                "relevant": false,
                "reason": "The statement provided does not have the needed context to help answer the question"
            }}
            ~~~
            ```
            Question: {question}
            ```
            Statement:{statement}
            ```
        '''.replace('            ','')


    def _generate_prompt(self, question: str, statement: str) -> str:
        return f'''
            Your role is to label chunks of text (statements) to determine whether they are relevant to a given question for a retrieval system.
            The statement and question are listed below (delimeted by ```).
            The statement provided is taken from employees of the company for which a question is asked. 
            A statement should be labeled as relevant (true) if it provides information that helps answer the question.
            It does not need to answer the question directly, but it should provide at least something relevant to the question.
            For example, if the question is "What do employees have to say about promotions?", a statement like 
            "I have never gotten a raise" would be labeled as relevant.
            Please format the output as a dictionary with the following keys: "relevant", "reason". 
            Relevant should be a boolean value indicating whether the statement is relevant to the question.
            Reason should be a string explaining why and how the statement is or is not relevant.
            An example and accompanying output is provided here (delimited by ~~~):
            ~~~
            Example:
            Question: "What team has won the most Super Bowls?"
            Statement: "I do not find that interesting"
            Output: {{
                "relevant": false,
                "reason": "The statement provided does not have the needed context to help answer the question"
            }}
            ~~~
            ```
            Question: {question}
            ```
            Statement:{statement}
            ```
        '''.replace('            ','')

##### Class Definition

In [37]:
import json
from langchain_community.llms import Ollama



class Evaluator:

    def __init__(self, llm: Ollama):
        self.llm = llm
        
    def _generate_prompt(self, question: str, statement: str) -> str:
        return f'''
            <s>[INST] <<SYS>>
            Your role is to label chunks of text (statements) to determine whether they are relevant to a given question for a retrieval system.
            The statement and question are listed below (delimeted by ```).
            The statement provided is taken from employees of the company for which a question is asked. 
            A statement should be labeled as relevant (true) if it provides information that helps answer the question.
            It does not need to answer the question directly, but it should provide at least something relevant to the question.
            A rewording of the question without adding any personal perspective should be considered unrelevant (false).
            For example, if the question is "What do employees have to say about promotions?", a statement like 
            "I have never gotten a raise" would be labeled as relevant.
            Please format the output as a dictionary with the following keys: "relevant", "reason". 
            Relevant should be a boolean value indicating whether the statement is relevant to the question.
            Reason should be a string explaining why and how the statement is or is not relevant.
            An example and accompanying output is provided here (delimited by ~~~):
            ~~~
            Example:
            Question: "What team has won the most Super Bowls?"
            Statement: "I do not find that interesting"
            Output: {{
                "relevant": false,
                "reason": "The statement provided does not have the needed context to help answer the question"
            }}
            ~~~
            <</SYS>>
            ```
            Question: {question}
            ```
            Statement:{statement}
            ```
            [/INST]
        '''.replace('            ','')
    
    def _format_output(self, output: str) -> dict:
        out_dict = {"output": output}
        output = output.replace('\n', '').replace('```','').strip()
        output = output[output.index('{'):output.rindex('}')+1]
        return json.loads(output) | out_dict

    def _invoke_llm(self, query: str) -> str:
        return self.llm.invoke(query)

    def evaluate_one(self, question, statement) -> dict:
        query = self._generate_prompt(question, statement).replace('\n', '')
        output = self._invoke_llm(query)
        try:
            return self._format_output(output)
        except:
            return {"relevant": None, "reason": "Error parsing output", "output": output}

## Example Prompt

In [20]:
evaluator = Evaluator(llm)
question = 'What do Best Buy employees think of the company?'
statement = 'I hate working here. The pay is terrible and the hours are long.'
print(evaluator._generate_prompt(question, statement))


<s>[INST] <<SYS>>
Your role is to label chunks of text (statements) to determine whether they are relevant to a given question for a retrieval system.
The statement and question are listed below (delimeted by ```).
The statement provided is taken from employees of the company for which a question is asked. 
A statement should be labeled as relevant (true) if it provides information that helps answer the question.
It does not need to answer the question directly, but it should provide at least something relevant to the question.
For example, if the question is "What do employees have to say about promotions?", a statement like 
"I have never gotten a raise" would be labeled as relevant.
Please format the output as a dictionary with the following keys: "relevant", "reason". 
Relevant should be a boolean value indicating whether the statement is relevant to the question.
Reason should be a string explaining why and how the statement is or is not relevant.
An example and accompanying outp

#### Test on two cases

In [19]:
evaluator = Evaluator(llm)
question = 'What do Best Buy employees think of the company?'

context1 = 'I hate working here. The pay is terrible and the hours are long.'
out1 = evaluator.evaluate_one(question, context1)

context2 = 'Why did you do that?'
out2 = evaluator.evaluate_one(question, context2)
print(f'Question: {question}\n')
print(f'Text 1: {context1}\nOutput: {out1}')
print(f'\nText 2: {context2}\nOutput: {out2}')

Question: What do Best Buy employees think of the company?

Text 1: I hate working here. The pay is terrible and the hours are long.
Output: {'relevant': True, 'reason': 'The statement provides some negative information about the company, which could be relevant to an inquiry into employee satisfaction or work environment.', 'output': '\n{\n"relevant": true,\n"reason": "The statement provides some negative information about the company, which could be relevant to an inquiry into employee satisfaction or work environment."\n}'}

Text 2: Why did you do that?
Output: {'relevant': False, 'reason': 'The statement provided does not relate to Best Buy or its employees, and therefore is not relevant to the question.', 'output': '\n{\n"relevant": false,\n"reason": "The statement provided does not relate to Best Buy or its employees, and therefore is not relevant to the question."\n}'}


In [25]:
import pandas as pd
pd.DataFrame([out1, out2])[['relevant','reason']]

,relevant,reason
0,True,The statement provides some negative informati...
1,False,The statement provided does not relate to Best...


#### Test reproducibility
1. With low temperature, we expect near identical outputs
2. With high temperature variance in the output is expected

In [17]:
evaluator = Evaluator(llm)
question = 'What do Best Buy employees think of the company?'
statement = 'I hate working here. The pay is terrible and the hours are long.'

outputs = []
for i in range(3):
    out = evaluator.evaluate_one(question, statement)
    outputs.append(out)
    print(out)

{'relevant': True, 'reason': 'The statement provides some negative information about the company, which could be relevant to an inquiry regarding employee opinions of Best Buy.', 'output': '\n{\n"relevant": true,\n"reason": "The statement provides some negative information about the company, which could be relevant to an inquiry regarding employee opinions of Best Buy."\n}'}
{'relevant': True, 'reason': 'The statement provides some negative information about the company, which could be relevant to an inquiry regarding employee opinions of Best Buy.', 'output': '\n{\n"relevant": true,\n"reason": "The statement provides some negative information about the company, which could be relevant to an inquiry regarding employee opinions of Best Buy."\n}'}
{'relevant': True, 'reason': 'The statement provides some negative information about the company, which could be relevant to an inquiry regarding employee opinions of Best Buy.', 'output': '\n{\n"relevant": true,\n"reason": "The statement provi

#### Running on samples of chunks

In [28]:
samples_df = pd.read_csv('../../data/best_buy/sample_questions_statements.csv')

In [29]:
samples_df

,question,statement
0,What do Best Buy employees think fo the company?,What is your opinion on Best Buy currently.
1,What do Best Buy employees think fo the company?,Why are you at Best Buy?
2,What do Best Buy employees think fo the company?,"Don’t listen to this guy, I work there and the..."
3,What do Best Buy employees think fo the company?,"Being a veteran and Best Buy employee, I can s..."
4,What do Best Buy employees think fo the company?,Life as a Best Buy worker 💀
...,...,...
114,Do employees feel understaffed?,Still insane to me they don’t do an annual 75%...
115,Do employees feel understaffed?,Hope you all can help - being denied refund; H...
116,Do employees feel understaffed?,New hire that still needs to receive paperwork...
117,Do employees feel understaffed?,Love this!!! My team was the same although si...


In [38]:
evaluator = Evaluator(llm)
sample_data = []
for i, row in samples_df.iterrows():
    question = row['question']
    statement = row['statement']
    out = evaluator.evaluate_one(question, statement)
    sample_data.append((out | row.to_dict()))
    print(f'\nQuestion: {question}')
    print(f'Statement: {statement}')
    print(f'Output: {out}')
    pd.DataFrame(sample_data).to_csv('../../data/best_buy/labelled_samples.csv', index=False)

#### Using ragas evaluate

In [ ]:
from datasets import Dataset 
from ragas.metrics import context_precision
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

In [11]:
score = evaluate(dataset,metrics=[context_precision], llm=llm)
score.to_pandas()

Evaluating: 100%|██████████| 2/2 [03:03<00:00, 91.95s/it]


,question,answer,contexts,ground_truth,context_precision
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967",[The First AFL–NFL World Championship Game was...,"The first superbowl was held on January 15, 1967",1.0
1,Who won the most super bowls?,The most super bowls have been won by The New ...,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...,0.0
